In [2]:
import pandas as pd
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import itertools

# adjusting display options
pd.set_option('display.max_colwidth', None)

data = pd.read_csv('../Dataset/sample_submission.csv')
n_rows = data.shape[0]
data

,id,text
0,0,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
1,1,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and
2,2,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice
3,3,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
4,4,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle
5,5,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle


In [3]:
# Step 3: Define Function to Calculate Perplexity
def calculate_perplexity(text, model, tokenizer):
    encodings = tokenizer(text, return_tensors="pt")
    max_length = model.config.n_positions  # Max length GPT-2 can handle
    stride = max_length

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = i
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - begin_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100  # ignore loss for all but last trg_len tokens
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
        nlls.append(neg_log_likelihood)
    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl.item()

In [4]:
# Step 4: Load Pre-Trained GPT-2 Model and Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\mebub_9a7jdi8\.conda\envs\Kaggle\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mebub_9a7jdi8\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
# Step 5: Select a Sample Text
sample_text = data.loc[0, 'text']
print(f"Original Text: {sample_text}")

Original Text: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge


In [6]:
# Step 6: Calculate Perplexity for Original Text
original_perplexity = calculate_perplexity(sample_text, model, tokenizer)
print(f"Original Perplexity: {original_perplexity}")

Original Perplexity: 475.3303527832031


In [7]:
# Step 7: Generate Permutations and Find Best Perplexity
words = sample_text.split()
permutations = list(itertools.permutations(words))
best_perplexity = float("inf")
best_permutation = None

for perm in permutations:
    permuted_text = " ".join(perm)
    current_perplexity = calculate_perplexity(permuted_text, model, tokenizer)
    if current_perplexity < best_perplexity:
        best_perplexity = current_perplexity
        best_permutation = perm

KeyboardInterrupt: 

In [ ]:
# Step 8: Print Results
print(f"Best Permutation: {' '.join(best_permutation)}")
print(f"Best Perplexity: {best_perplexity}")